In [7]:
import pandas as pd
import numpy as np

np.random.seed(1212)

import keras
from keras.models import Model
from keras.layers import *
from keras import optimizers

In [9]:
df_train = pd.read_csv(r'C:\Users\91858\Desktop\python\mnist-in-csv\mnist_train.csv')
df_test = pd.read_csv(r'C:\Users\91858\Desktop\python\mnist-in-csv\mnist_test.csv')

In [12]:
df_train.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
df_features = df_train.iloc[:, 1:785]
df_label = df_train.iloc[:, 0]

X_test = df_test.iloc[:, 0:784]

print(X_test.shape)

(10000, 784)


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(df_features, df_label, 
                                                test_size = 0.25,
                                                random_state = 1212)

X_train = X_train.as_matrix().reshape(45000, 784) #(45000, 784)
X_cv = X_cv.as_matrix().reshape(15000, 784) #(15000, 784)

X_test = X_test.as_matrix().reshape(10000, 784)

C:\Users\91858\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\Users\91858\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
C:\Users\91858\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':


In [18]:
print((min(X_train[1]), max(X_train[1])))

(0, 254)


In [19]:
# Feature Normalization 
X_train = X_train.astype('float32'); X_cv= X_cv.astype('float32'); X_test = X_test.astype('float32')
X_train /= 255; X_cv /= 255; X_test /= 255

# Convert labels to One Hot Encoded
num_digits = 10
y_train = keras.utils.to_categorical(y_train, num_digits)
y_cv = keras.utils.to_categorical(y_cv, num_digits)

In [20]:
# Printing 2 examples of labels after conversion
print(y_train[0]) # 2
print(y_train[3]) # 7

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [21]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 200
num_digits = 10

In [22]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

In [23]:
# Our model would have '6' layers - input layer, 4 hidden layer and 1 output layer
model = Model(Inp, output)
model.summary() # We have 297,910 parameters to estimate

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 784)               0         
_________________________________________________________________
Hidden_Layer_1 (Dense)       (None, 300)               235500    
_________________________________________________________________
Hidden_Layer_2 (Dense)       (None, 100)               30100     
_________________________________________________________________
Hidden_Layer_3 (Dense)       (None, 100)               10100     
_________________________________________________________________
Hidden_Layer_4 (Dense)       (None, 200)               20200     
_________________________________________________________________
Output_Layer (Dense)         (None, 10)                2010      
Total params: 297,910
Trainable params: 297,910
Non-trainable params: 0
_____________________________________________________

In [24]:
# Insert Hyperparameters
learning_rate = 0.1
training_epochs = 20
batch_size = 100
sgd = optimizers.SGD(lr=learning_rate)

In [25]:
# We rely on the plain vanilla Stochastic Gradient Descent as our optimizing methodology
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [26]:
history1 = model.fit(X_train, y_train,
                     batch_size = batch_size,
                     epochs = training_epochs,
                     verbose = 2,
                     validation_data=(X_cv, y_cv))

Train on 45000 samples, validate on 15000 samples
Epoch 1/20
 - 2s - loss: 1.5993 - accuracy: 0.5708 - val_loss: 0.6925 - val_accuracy: 0.8155
Epoch 2/20
 - 2s - loss: 0.4989 - accuracy: 0.8615 - val_loss: 0.3932 - val_accuracy: 0.8876
Epoch 3/20
 - 2s - loss: 0.3546 - accuracy: 0.8994 - val_loss: 0.3217 - val_accuracy: 0.9045
Epoch 4/20
 - 2s - loss: 0.3005 - accuracy: 0.9141 - val_loss: 0.2762 - val_accuracy: 0.9183
Epoch 5/20
 - 2s - loss: 0.2656 - accuracy: 0.9235 - val_loss: 0.2484 - val_accuracy: 0.9263
Epoch 6/20
 - 2s - loss: 0.2383 - accuracy: 0.9314 - val_loss: 0.2290 - val_accuracy: 0.9327
Epoch 7/20
 - 3s - loss: 0.2178 - accuracy: 0.9371 - val_loss: 0.2100 - val_accuracy: 0.9368
Epoch 8/20
 - 2s - loss: 0.1990 - accuracy: 0.9427 - val_loss: 0.1952 - val_accuracy: 0.9422
Epoch 9/20
 - 2s - loss: 0.1833 - accuracy: 0.9470 - val_loss: 0.1815 - val_accuracy: 0.9455
Epoch 10/20
 - 2s - loss: 0.1705 - accuracy: 0.9507 - val_loss: 0.1809 - val_accuracy: 0.9461
Epoch 11/20
 - 2s -

In [27]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

# We rely on ADAM as our optimizing methodology
adam = keras.optimizers.Adam(lr=learning_rate)
model2 = Model(Inp, output)

model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [28]:
history2 = model2.fit(X_train, y_train,
                      batch_size = batch_size,
                      epochs = training_epochs,
                      verbose = 2,
                      validation_data=(X_cv, y_cv))

Train on 45000 samples, validate on 15000 samples
Epoch 1/20
 - 3s - loss: 0.2947 - accuracy: 0.9127 - val_loss: 0.1396 - val_accuracy: 0.9575
Epoch 2/20
 - 3s - loss: 0.1081 - accuracy: 0.9670 - val_loss: 0.1062 - val_accuracy: 0.9677
Epoch 3/20
 - 3s - loss: 0.0733 - accuracy: 0.9766 - val_loss: 0.0843 - val_accuracy: 0.9741
Epoch 4/20
 - 3s - loss: 0.0526 - accuracy: 0.9833 - val_loss: 0.1016 - val_accuracy: 0.9711
Epoch 5/20
 - 3s - loss: 0.0425 - accuracy: 0.9861 - val_loss: 0.0939 - val_accuracy: 0.9723
Epoch 6/20
 - 3s - loss: 0.0314 - accuracy: 0.9897 - val_loss: 0.0832 - val_accuracy: 0.9773
Epoch 7/20
 - 3s - loss: 0.0293 - accuracy: 0.9900 - val_loss: 0.0963 - val_accuracy: 0.9741
Epoch 8/20
 - 3s - loss: 0.0290 - accuracy: 0.9910 - val_loss: 0.0997 - val_accuracy: 0.9757
Epoch 9/20
 - 3s - loss: 0.0197 - accuracy: 0.9932 - val_loss: 0.0951 - val_accuracy: 0.9762
Epoch 10/20
 - 3s - loss: 0.0208 - accuracy: 0.9930 - val_loss: 0.0922 - val_accuracy: 0.9777
Epoch 11/20
 - 3s -

In [29]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

learning_rate = 0.01
adam = keras.optimizers.Adam(lr=learning_rate)
model2a = Model(Inp, output)

model2a.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [30]:
history2a = model2a.fit(X_train, y_train,
                        batch_size = batch_size,
                        epochs = training_epochs,
                        verbose = 2,
                        validation_data=(X_cv, y_cv))

Train on 45000 samples, validate on 15000 samples
Epoch 1/20
 - 3s - loss: 0.2978 - accuracy: 0.9113 - val_loss: 0.1705 - val_accuracy: 0.9489
Epoch 2/20
 - 3s - loss: 0.1124 - accuracy: 0.9663 - val_loss: 0.1017 - val_accuracy: 0.9689
Epoch 3/20
 - 4s - loss: 0.0748 - accuracy: 0.9762 - val_loss: 0.0826 - val_accuracy: 0.9744
Epoch 4/20
 - 4s - loss: 0.0540 - accuracy: 0.9822 - val_loss: 0.0910 - val_accuracy: 0.9750
Epoch 5/20
 - 4s - loss: 0.0438 - accuracy: 0.9858 - val_loss: 0.0944 - val_accuracy: 0.9737
Epoch 6/20
 - 4s - loss: 0.0371 - accuracy: 0.9878 - val_loss: 0.0845 - val_accuracy: 0.9763
Epoch 7/20
 - 3s - loss: 0.0272 - accuracy: 0.9912 - val_loss: 0.0995 - val_accuracy: 0.9727
Epoch 8/20
 - 3s - loss: 0.0253 - accuracy: 0.9914 - val_loss: 0.0875 - val_accuracy: 0.9763
Epoch 9/20
 - 3s - loss: 0.0265 - accuracy: 0.9912 - val_loss: 0.0789 - val_accuracy: 0.9809
Epoch 10/20
 - 3s - loss: 0.0199 - accuracy: 0.9934 - val_loss: 0.0977 - val_accuracy: 0.9761
Epoch 11/20
 - 3s -

In [31]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

learning_rate = 0.5
adam = keras.optimizers.Adam(lr=learning_rate)
model2b = Model(Inp, output)

model2b.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [32]:
history2b = model2b.fit(X_train, y_train,
                        batch_size = batch_size,
                        epochs = training_epochs,
                            validation_data=(X_cv, y_cv))

Train on 45000 samples, validate on 15000 samples
Epoch 1/20
45000/45000 [==============================] - 4s 79us/step - loss: 0.3057 - accuracy: 0.9095 - val_loss: 0.1215 - val_accuracy: 0.9635
Epoch 2/20
45000/45000 [==============================] - 3s 74us/step - loss: 0.1107 - accuracy: 0.9666 - val_loss: 0.1250 - val_accuracy: 0.9622
Epoch 3/20
45000/45000 [==============================] - 3s 72us/step - loss: 0.0737 - accuracy: 0.9771 - val_loss: 0.0790 - val_accuracy: 0.9759
Epoch 4/20
45000/45000 [==============================] - 3s 72us/step - loss: 0.0514 - accuracy: 0.9840 - val_loss: 0.0892 - val_accuracy: 0.9748
Epoch 5/20
45000/45000 [==============================] - 3s 73us/step - loss: 0.0441 - accuracy: 0.9859 - val_loss: 0.0832 - val_accuracy: 0.9765
Epoch 6/20
45000/45000 [==============================] - 3s 75us/step - loss: 0.0336 - accuracy: 0.9888 - val_loss: 0.0887 - val_accuracy: 0.9765
Epoch 7/20
45000/45000 [==============================] - 3s 76us/st

In [33]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 100
n_hidden_5 = 200
num_digits = 10

In [34]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
x = Dense(n_hidden_5, activation='relu', name = "Hidden_Layer_5")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

In [35]:
# Our model would have '7' layers - input layer, 5 hidden layer and 1 output layer
model3 = Model(Inp, output)
model3.summary() # We have 308,010 parameters to estimate

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 784)               0         
_________________________________________________________________
Hidden_Layer_1 (Dense)       (None, 300)               235500    
_________________________________________________________________
Hidden_Layer_2 (Dense)       (None, 100)               30100     
_________________________________________________________________
Hidden_Layer_3 (Dense)       (None, 100)               10100     
_________________________________________________________________
Hidden_Layer_4 (Dense)       (None, 100)               10100     
_________________________________________________________________
Hidden_Layer_5 (Dense)       (None, 200)               20200     
_________________________________________________________________
Output_Layer (Dense)         (None, 10)                2010

In [36]:
# We rely on 'Adam' as our optimizing methodology
adam = keras.optimizers.Adam(lr=0.01)

model3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [37]:
history3 = model3.fit(X_train, y_train,
                      batch_size = batch_size,
                      epochs = training_epochs,
                      validation_data=(X_cv, y_cv))

Train on 45000 samples, validate on 15000 samples
Epoch 1/20
45000/45000 [==============================] - 4s 79us/step - loss: 0.3054 - accuracy: 0.9077 - val_loss: 0.1350 - val_accuracy: 0.9583
Epoch 2/20
45000/45000 [==============================] - 3s 74us/step - loss: 0.1131 - accuracy: 0.9649 - val_loss: 0.1002 - val_accuracy: 0.9707
Epoch 3/20
45000/45000 [==============================] - 3s 74us/step - loss: 0.0757 - accuracy: 0.9762 - val_loss: 0.1076 - val_accuracy: 0.9680
Epoch 4/20
45000/45000 [==============================] - 3s 74us/step - loss: 0.0583 - accuracy: 0.9816 - val_loss: 0.1058 - val_accuracy: 0.9691
Epoch 5/20
45000/45000 [==============================] - 3s 74us/step - loss: 0.0453 - accuracy: 0.9853 - val_loss: 0.0883 - val_accuracy: 0.9757
Epoch 6/20
45000/45000 [==============================] - 4s 84us/step - loss: 0.0387 - accuracy: 0.9870 - val_loss: 0.0915 - val_accuracy: 0.9763
Epoch 7/20
45000/45000 [==============================] - 3s 73us/st

In [38]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 200
num_digits = 10

In [39]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dropout(0.3)(x)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dropout(0.3)(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dropout(0.3)(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

In [40]:
# Our model would have '6' layers - input layer, 4 hidden layer and 1 output layer
model4 = Model(Inp, output)
model4.summary() # We have 297,910 parameters to estimate

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 784)               0         
_________________________________________________________________
Hidden_Layer_1 (Dense)       (None, 300)               235500    
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
Hidden_Layer_2 (Dense)       (None, 100)               30100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
Hidden_Layer_3 (Dense)       (None, 100)               10100     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0   

In [41]:
model4.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [42]:
history = model4.fit(X_train, y_train,
                    batch_size = batch_size,
                    epochs = training_epochs,
                    validation_data=(X_cv, y_cv))

Train on 45000 samples, validate on 15000 samples
Epoch 1/20
45000/45000 [==============================] - 5s 101us/step - loss: 0.5116 - accuracy: 0.8385 - val_loss: 0.1647 - val_accuracy: 0.9498
Epoch 2/20
45000/45000 [==============================] - 4s 84us/step - loss: 0.2165 - accuracy: 0.9380 - val_loss: 0.1178 - val_accuracy: 0.9647
Epoch 3/20
45000/45000 [==============================] - 4s 81us/step - loss: 0.1610 - accuracy: 0.9532 - val_loss: 0.0990 - val_accuracy: 0.9713
Epoch 4/20
45000/45000 [==============================] - 4s 80us/step - loss: 0.1317 - accuracy: 0.9613 - val_loss: 0.0925 - val_accuracy: 0.9735
Epoch 5/20
45000/45000 [==============================] - 4s 87us/step - loss: 0.1179 - accuracy: 0.9653 - val_loss: 0.0889 - val_accuracy: 0.9748
Epoch 6/20
45000/45000 [==============================] - 4s 87us/step - loss: 0.1071 - accuracy: 0.9690 - val_loss: 0.0895 - val_accuracy: 0.9760
Epoch 7/20
45000/45000 [==============================] - 4s 85us/s

In [43]:
test_pred = pd.DataFrame(model4.predict(X_test, batch_size=200))
test_pred = pd.DataFrame(test_pred.idxmax(axis = 1))
test_pred.index.name = 'ImageId'
test_pred = test_pred.rename(columns = {0: 'Label'}).reset_index()
test_pred['ImageId'] = test_pred['ImageId'] + 1

test_pred.head()

,ImageId,Label
0,1,7
1,2,2
2,3,1
3,4,0
4,5,4


In [44]:
test_pred.to_csv('mnist_submission.csv', index = False)